In [1]:
# install dependencies
!pip install diffusers
!pip install transformers

In [ ]:
from diffusers import StableDiffusionPipeline
import torch

model_id = "runwayml/stable-diffusion-v1-5"
sd_pipeline = StableDiffusionPipeline.from_pretrained(model_id,
                                                      torch_dtype=torch.float16)
sd_pipeline = sd_pipeline.to("cuda")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

In [ ]:
def get_completion_sd(prompt):
  negative_prompt = """
  simple background, duplicate, low quality, lowest quality,
  bad anatomy, bad proportions, extra digits, lowres, username,
  artist name, error, duplicate, watermark, signature, text,
  extra digit, fewer digits, worst quality, jpeg artifacts, blurry
  """
  return sd_pipeline(prompt, negative_prompt=negative_prompt).images[0]

In [ ]:
#let's prompt

# prompt = "astronaut, riding a horse, on mars, human colony"
# prompt = "children, playing in disneyland, view from a distance"
prompt = """baby llama, wearing red muffler,
grazing, open field, sunset
"""
print(prompt)
sd_image = get_completion_sd(prompt)
sd_image.save("./llama.jpg")

baby llama, wearing red muffler,
grazing, open field, sunset



  0%|          | 0/50 [00:00<?, ?it/s]

### Build AI Image Generator

In [ ]:
!pip install gradio==3.48.0

In [ ]:
import gradio as gr

In [ ]:
def get_completion(category, style, color_scheme, pattern, material, fit, occasion, season):
    PROMPT_TEMPLATE = """A professional fashion model wearing a {category}, designed in {style} style with {color_scheme} colors and a {pattern} pattern. The outfit is crafted from {material}, featuring a {fit} fit, perfect for {occasion} and ideal for {season} wear. The model has a smooth, mannequin-like face with subtle contours, minimal facial details, and a neutral expression, ensuring a fashion-focused presentation. Captured in a high-end fashion studio with professional lighting, this ultra-realistic 8K UHD image highlights the outfit’s intricate textures and design. The model poses confidently against a pure white background, maintaining a clean and professional aesthetic."""

    prompt = PROMPT_TEMPLATE.format(
        category=category, style=style, color_scheme=color_scheme,
        pattern=pattern, material=material, fit=fit,
        occasion=occasion, season=season
    )

    negative_prompt = """
    simple background, duplicate, low quality, lowest quality,
    bad anatomy, bad proportions, extra digits, lowres, username,
    artist name, error, duplicate, watermark, signature, text,
    extra digit, fewer digits, worst quality, jpeg artifacts, blurry face
    """

    return sd_pipeline(prompt, negative_prompt=negative_prompt).images[0]

categories = ["Dress", "T-shirt", "Jeans", "Jacket","Shirt and Jeans","Lehenga","Churidar","Saree","Gown"]
styles = ["Modern", "Vintage", "Casual", "Elegant","Traditional"]
color_schemes = ["Monochrome", "Pastel", "Vibrant"]
patterns = ["Striped", "Floral", "Plain","Embroidery"]
materials = ["Cotton", "Silk", "Denim", "Leather","Georgette","Crepe"]
fits = ["Slim", "Regular", "Loose"]
occasions = ["Formal", "Casual", "Party"]
seasons = ["Summer", "Winter","Normal"]

genai_app = gr.Interface(
    fn=get_completion,
    inputs=[
        gr.Dropdown(choices=categories, label="Category"),
        gr.Dropdown(choices=styles, label="Style"),
        gr.Dropdown(choices=color_schemes, label="Color Scheme"),
        gr.Dropdown(choices=patterns, label="Pattern"),
        gr.Dropdown(choices=materials, label="Material"),
        gr.Dropdown(choices=fits, label="Fit"),
        gr.Dropdown(choices=occasions, label="Occasion"),
        gr.Dropdown(choices=seasons, label="Season")
    ],
    outputs=[gr.Image(label="Generated Image")],
    title="Fashion AI Image Generator",
    description="Select options to generate a fashion photography image using Stable Diffusion.",
    allow_flagging="never"
)

genai_app.launch()


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
IMPORTANT: You are using gradio version 3.48.0, however version 4.44.1 is available, please upgrade.
--------
Running on public URL: https://9448044684d4123b16.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [ ]:
pip install --upgrade gradio


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 MB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.1/322.1 kB 27.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 115.3 MB/s eta 0:00:00
  Attempting uninstall: gradio-client
    Found existing installation: gradio_client 0.6.1
    Uninstalling gradio_client-0.6.1:
      Successfully uninstalled gradio_client-0.6.1
  Attempting uninstall: gradio
    Found existing installation: gradio 3.48.0
    Uninstalling gradio-3.48.0:
      Successfully uninstalled gradio-3.48.0


In [ ]:
import cv2
import numpy as np
import gradio as gr
from diffusers import StableDiffusionPipeline
from PIL import Image

# Load Stable Diffusion model
sd_pipeline = StableDiffusionPipeline.from_pretrained("runwayml/stable-diffusion-v1-5").to("cuda")

def overlay_face(fashion_image, face_image):
    if isinstance(face_image, str):  # If face image is a file path
        face_image = cv2.imread(face_image)

    if face_image is None or not isinstance(face_image, np.ndarray):
        print("Invalid face image input!")
        return fashion_image  # Return the original fashion image

    # Convert face image to grayscale
    gray_face = cv2.cvtColor(face_image, cv2.COLOR_BGR2GRAY)

    # Load pre-trained face detector
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

    # Detect faces in the face image
    faces = face_cascade.detectMultiScale(gray_face, scaleFactor=1.3, minNeighbors=5)

    if len(faces) == 0:
        print("No face detected!")
        return fashion_image  # Return the original if no face is found

    x, y, w, h = faces[0]  # Get the first detected face
    face_crop = face_image[y:y+h, x:x+w]  # Crop the detected face

    # Convert fashion image from PIL to NumPy if needed
    if isinstance(fashion_image, Image.Image):
        fashion_image = np.array(fashion_image)

    # Convert to grayscale for face detection on fashion image
    gray_fashion = cv2.cvtColor(fashion_image, cv2.COLOR_BGR2GRAY)

    # Detect face in the generated fashion image
    model_faces = face_cascade.detectMultiScale(gray_fashion, scaleFactor=1.1, minNeighbors=5)

    if len(model_faces) == 0:
        print("No face detected on model!")
        return fashion_image

    # Get the model's face position
    mx, my, mw, mh = model_faces[0]

    # Resize the face to fit the model's face size
    face_resized = cv2.resize(face_crop, (mw, mh))

    # Blend the face onto the model's face
    fashion_image[my:my+mh, mx:mx+mw] = face_resized

    return fashion_image

def get_completion(category, style, color_scheme, pattern, material, fit, occasion, season, face_image):
    PROMPT_TEMPLATE = """A professional fashion model wearing a {category}, designed in {style} style with {color_scheme} colors and a {pattern} pattern. The outfit is crafted from {material}, featuring a {fit} fit, perfect for {occasion} and ideal for {season} wear. The model has a smooth, mannequin-like face with subtle contours, minimal facial details, and a neutral expression, ensuring a fashion-focused presentation. Captured in a high-end fashion studio with professional lighting, this ultra-realistic 8K UHD image highlights the outfit’s intricate textures and design. The model poses confidently against a pure white background, maintaining a clean and professional aesthetic."""

    prompt = PROMPT_TEMPLATE.format(
        category=category, style=style, color_scheme=color_scheme,
        pattern=pattern, material=material, fit=fit,
        occasion=occasion, season=season
    )

    # Generate fashion model image
    generated_image = sd_pipeline(prompt, guidance_scale=7.5).images[0]

    # Overlay face if provided
    if isinstance(face_image, np.ndarray) or isinstance(face_image, str):
        generated_image = overlay_face(generated_image, face_image)

    # Convert back to PIL image
    if isinstance(generated_image, np.ndarray):
        generated_image = Image.fromarray(generated_image)

    return generated_image

# Dropdown options
categories = ["Dress", "T-shirt", "Jeans", "Jacket", "Shirt and Jeans", "Lehenga", "Churidar", "Saree", "Gown"]
styles = ["Modern", "Vintage", "Casual", "Elegant", "Traditional"]
color_schemes = ["Monochrome", "Pastel", "Vibrant"]
patterns = ["Striped", "Floral", "Plain", "Embroidery"]
materials = ["Cotton", "Silk", "Denim", "Leather", "Georgette", "Crepe"]
fits = ["Slim", "Regular", "Loose"]
occasions = ["Formal", "Casual", "Party"]
seasons = ["Summer", "Winter", "Normal"]

# Gradio UI
genai_app = gr.Interface(
    fn=get_completion,
    inputs=[
        gr.Dropdown(choices=categories, label="Category"),
        gr.Dropdown(choices=styles, label="Style"),
        gr.Dropdown(choices=color_schemes, label="Color Scheme"),
        gr.Dropdown(choices=patterns, label="Pattern"),
        gr.Dropdown(choices=materials, label="Material"),
        gr.Dropdown(choices=fits, label="Fit"),
        gr.Dropdown(choices=occasions, label="Occasion"),
        gr.Dropdown(choices=seasons, label="Season"),
        gr.Image(type="filepath", label="Upload Your Face")  # Ensure correct input type
    ],
    outputs=gr.Image(label="Generated Image with Your Face"),
    title="Fashion AI Image Generator",
    description="Select options to generate a fashion photography image using Stable Diffusion, with an option to overlay your face onto the model.",
    allow_flagging="never"
)

genai_app.launch()


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
IMPORTANT: You are using gradio version 3.48.0, however version 4.44.1 is available, please upgrade.
--------
Running on public URL: https://810d9235b0e58c4bce.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [ ]:
import cv2
import numpy as np
import gradio as gr
from diffusers import StableDiffusionPipeline
from PIL import Image

# Load Stable Diffusion model
sd_pipeline = StableDiffusionPipeline.from_pretrained("runwayml/stable-diffusion-v1-5").to("cuda")

def apply_generated_clothing(uploaded_image, category, style, color_scheme, pattern, material, fit, occasion, season):
    PROMPT_TEMPLATE = """A professional fashion model wearing a {category}, designed in {style} style with {color_scheme} colors and a {pattern} pattern. The outfit is crafted from {material}, featuring a {fit} fit, perfect for {occasion} and ideal for {season} wear. The model has a smooth, mannequin-like face with subtle contours, minimal facial details, and a neutral expression, ensuring a fashion-focused presentation. Captured in a high-end fashion studio with professional lighting, this ultra-realistic 8K UHD image highlights the outfit’s intricate textures and design. The model poses confidently against a pure white background, maintaining a clean and professional aesthetic."""

    prompt = PROMPT_TEMPLATE.format(
        category=category, style=style, color_scheme=color_scheme,
        pattern=pattern, material=material, fit=fit,
        occasion=occasion, season=season
    )

    # Generate only the clothing
    generated_image = sd_pipeline(prompt, guidance_scale=7.5).images[0]

    # Convert images to NumPy arrays for processing
    uploaded_np = np.array(uploaded_image)
    generated_np = np.array(generated_image)

    # Extract clothing from the generated image
    gray_generated = cv2.cvtColor(generated_np, cv2.COLOR_BGR2GRAY)
    _, mask = cv2.threshold(gray_generated, 128, 255, cv2.THRESH_BINARY)

    # Blend the clothing onto the uploaded image
    result = uploaded_np.copy()
    result[mask > 0] = generated_np[mask > 0]  # Apply clothing while keeping the face

    return Image.fromarray(result)

# Dropdown options
categories = ["Dress", "T-shirt", "Jeans", "Jacket", "Shirt and Jeans", "Lehenga", "Churidar", "Saree", "Gown"]
styles = ["Modern", "Vintage", "Casual", "Elegant", "Traditional"]
color_schemes = ["Monochrome", "Pastel", "Vibrant"]
patterns = ["Striped", "Floral", "Plain", "Embroidery"]
materials = ["Cotton", "Silk", "Denim", "Leather", "Georgette", "Crepe"]
fits = ["Slim", "Regular", "Loose"]
occasions = ["Formal", "Casual", "Party"]
seasons = ["Summer", "Winter", "Normal"]

# Gradio UI
genai_app = gr.Interface(
    fn=apply_generated_clothing,
    inputs=[
        gr.Image(type="pil", label="Upload Your Image"),
        gr.Dropdown(choices=categories, label="Category"),
        gr.Dropdown(choices=styles, label="Style"),
        gr.Dropdown(choices=color_schemes, label="Color Scheme"),
        gr.Dropdown(choices=patterns, label="Pattern"),
        gr.Dropdown(choices=materials, label="Material"),
        gr.Dropdown(choices=fits, label="Fit"),
        gr.Dropdown(choices=occasions, label="Occasion"),
        gr.Dropdown(choices=seasons, label="Season"),
    ],
    outputs=gr.Image(label="Generated Image with Your Clothing"),
    title="Fashion AI Image Generator",
    description="Upload your image, select fashion options, and apply the generated clothing while keeping your face.",
    allow_flagging="never"
)

genai_app.launch()


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
IMPORTANT: You are using gradio version 3.48.0, however version 4.44.1 is available, please upgrade.
--------
Running on public URL: https://0f3215161762aa3a66.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
